In [1]:
import pandas as pd
import geopandas as gpd

# Datos OSM

In [3]:
# importar capas
gdf_parks = gpd.read_file("../1_Datos/OSM/leisure_park_pol.geojson")
gdf_grass = gpd.read_file("../1_Datos/OSM/landuse_grass_pol.geojson")
gdf_grassland = gpd.read_file("../1_Datos/OSM/natural_grassland_pol.geojson")
gdf_forest = gpd.read_file("../1_Datos/OSM/landuse_forest_pol.geojson")
gdf_wood = gpd.read_file("../1_Datos/OSM/natural_wood_pol.geojson")
gdf_reserve = gpd.read_file("../1_Datos/OSM/leisure_naturereserve_pol.geojson")

In [10]:
# generación de campo categoría
gdf_parks['categoria'] = 'park'
gdf_grass['categoria'] = 'grass'
gdf_grassland['categoria'] = 'grassland'
gdf_forest['categoria'] = 'forest'
gdf_wood['categoria'] = 'wood'
gdf_reserve['categoria'] = 'naturereserve'

In [14]:
# concatenación de los gdfs
gdf_osm = pd.concat([gdf_parks, gdf_grass, gdf_grassland, gdf_forest, gdf_wood, gdf_reserve])
# generación del campo fuente y epv
gdf_osm['fuente'] = 1
gdf_osm['epv'] = 1
# filtro de las columnas de interés
gdf_osm = gdf_osm[['full_id', 'osm_id', 'leisure', 'epv', 'categoria', 'name', 'fuente', 'geometry']]

In [15]:
gdf_osm

,full_id,osm_id,leisure,epv,categoria,name,fuente,geometry
0,r456535,456535,park,1,park,Plaza de la Constitución,1,"MULTIPOLYGON (((-58.38040 -34.62623, -58.38039..."
1,r533095,533095,park,1,park,Parque de los Patricios,1,"MULTIPOLYGON (((-58.40904 -34.63808, -58.40898..."
2,r533112,533112,park,1,park,Parque Leonardo Pereyra,1,"MULTIPOLYGON (((-58.38899 -34.64975, -58.38845..."
3,r1775982,1775982,park,1,park,,1,"MULTIPOLYGON (((-58.85579 -34.63345, -58.85604..."
4,r1822808,1822808,park,1,park,Plaza General Las Heras,1,"MULTIPOLYGON (((-58.94748 -34.92715, -58.94710..."
...,...,...,...,...,...,...,...,...
0,r10343154,10343154,nature_reserve,1,naturereserve,Reserva Ecológica Costanera Sur,1,"MULTIPOLYGON (((-58.36301 -34.59783, -58.36271..."
1,r11925488,11925488,nature_reserve,1,naturereserve,Parque Ecológico Municipal,1,"MULTIPOLYGON (((-58.06319 -34.86334, -58.06270..."
2,w544835543,544835543,nature_reserve,1,naturereserve,,1,"MULTIPOLYGON (((-58.58105 -34.63511, -58.58096..."
3,w583565805,583565805,nature_reserve,1,naturereserve,Reserva Natural de Ciudad Evita,1,"MULTIPOLYGON (((-58.53723 -34.74592, -58.53876..."


# IGN

In [17]:
# importar capa
gdf_ign = gpd.read_file("../1_Datos/IGN/EspaciosVerdes.geojson")

# Filtro de las capas al área de interés

In [70]:
# importar capas de municipios y límite antrópico
gdf_muni = gpd.read_file("../1_Datos/ACUMAR/LIMITE_POLITICO.shp")
gdf_limite = gpd.read_file("../1_Datos/ACUMAR/LIMITE_ANTROPICO_POLIGONO.geojson")

In [37]:
# filtro de los municipios de interés 
gdf_muni = gdf_muni.loc[(gdf_muni.DISTRITO == "LOMAS DE ZAMORA") | (gdf_muni.DISTRITO == "LANUS")]

In [62]:
gdf_osm_muni = gpd.sjoin(gdf_osm, gdf_muni, how='left')
gdf_osm_muni = gdf_osm_muni.loc[gdf_osm_muni.index_right.notnull()]
gdf_osm_muni = gdf_osm_muni[['full_id', 'osm_id', 'leisure', 'epv', 'categoria', 'name', 'fuente', 'DISTRITO', 'geometry']]
print("Polígonos OSM únicos:", len(gdf_osm_muni.full_id.unique()) == len(gdf_osm_muni))
gdf_ign_muni = gpd.sjoin(gdf_ign, gdf_muni, how='left')
gdf_ign_muni = gdf_ign_muni.loc[gdf_ign_muni.index_right.notnull()]
gdf_ign_muni = gdf_ign_muni[['gid', 'entidad', 'objeto', 'fna', 'gna', 'nam', 'fdc', 'sag', 'DISTRITO', 'geometry']]
print("Polígonos IGN únicos:", len(gdf_ign_muni.gid.unique()) == len(gdf_ign_muni))

Polígonos OSM únicos: True
Polígonos IGN únicos: True


In [77]:
gdf_osm_lim = gpd.sjoin(gdf_osm_muni, gdf_limite, how='left')
gdf_osm_lim['limite'] = 1
gdf_osm_lim.loc[gdf_osm_lim.index_right.isnull(), 'limite'] = 0
gdf_osm_lim = gdf_osm_lim[['full_id', 'osm_id', 'leisure', 'epv', 'categoria', 'name', 'fuente', 'DISTRITO', 'limite', 'geometry']]
gdf_ign_lim = gpd.sjoin(gdf_ign_muni, gdf_limite, how='left')
gdf_ign_lim['limite'] = 1
gdf_ign_lim.loc[gdf_ign_lim.index_right.isnull(), 'limite'] = 0
gdf_ign_lim = gdf_ign_lim[['gid', 'entidad', 'objeto', 'fna', 'gna', 'nam', 'fdc', 'sag', 'DISTRITO', 'limite', 'geometry']]

In [79]:
gdf_osm_lim.to_file("./Capas_Output/OSM_epv.geojson", driver="GeoJSON")
gdf_ign_lim.to_file("./Capas_Output/IGM_epv.geojson", driver="GeoJSON")

# Registros OSM superpuestos

In [123]:
gdf_osm = gpd.read_file("./Capas_Output/OSM_epv.geojson")
gdf_ign = gpd.read_file("./Capas_Output/IGM_epv.geojson")

In [124]:
# Registro OSM superpuestos entre si
gdf_osm['superpuesto'] = 0

for i in range(len(gdf_osm)):
    gdf_registrosuperpuesto = gpd.sjoin(gdf_osm.loc[[i]], gdf_osm.loc[gdf_osm.index != i], how='left')

    if len(gdf_registrosuperpuesto.loc[gdf_registrosuperpuesto.full_id_right.notnull()]) > 0:
        gdf_osm.loc[i, 'superpuesto'] = 1

print("Polígonos sin superposición", len(gdf_osm.loc[gdf_osm['superpuesto']==0]))
print("Polígonos con superposición", len(gdf_osm.loc[gdf_osm['superpuesto']==1]))

Polígonos sin superposición 209
Polígonos con superposición 20


# Cruce de capas

In [169]:
# Cruso capas y me quedo con las que no se superponen
gdf_ign_sjoin = gpd.sjoin(gdf_ign, gdf_osm, how='left')
print("Parcelas IGN:", len(gdf_ign_sjoin.loc[(gdf_ign_sjoin.index_right.isnull())]))
gdf_ign_sjoin = gdf_ign_sjoin[['gid', 'entidad', 'objeto', 'fna', 'gna', 'nam', 'fdc', 'sag',
       'DISTRITO_left', 'limite_left', 'geometry', 'index_right']]
gdf_ign_sjoin.rename(columns={'DISTRITO_left':'DISTRITO'}, inplace=True)
gdf_ign_sjoin.loc[(gdf_ign_sjoin.index_right.isnull())].to_file("./Capas_Output/IGN_sinintersect.geojson", driver="GeoJSON")

gdf_osm_sjoin = gpd.sjoin(gdf_osm, gdf_ign, how='left')
print("Parcelas OSM:", len(gdf_osm_sjoin.loc[(gdf_osm_sjoin.index_right.isnull())]))
gdf_osm_sjoin = gdf_osm_sjoin[['full_id', 'osm_id', 'leisure', 'epv', 'categoria', 'name', 'fuente',
       'DISTRITO_left', 'limite_left', 'geometry', 'superpuesto', 'index_right']]
gdf_osm_sjoin.rename(columns={'DISTRITO_left':'DISTRITO'}, inplace=True)
gdf_osm_sjoin.loc[(gdf_osm_sjoin.index_right.isnull())].to_file("./Capas_Output/OSM_sinintersect.geojson", driver="GeoJSON")

Parcelas IGN: 5
Parcelas OSM: 107


In [128]:
# cruce entre capas y seleccionar una intersección
gdf_ign['area_ign'] = gdf_ign.area
gdf_osm['area_osm'] = gdf_osm.area
gdf_ignosm_sjoin = gpd.sjoin(gdf_ign, gdf_osm, how='left')

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until


In [153]:
""" gdf_ignosm_super = gdf_ignosm_sjoin.loc[(gdf_ignosm_sjoin.index_right.notnull())]
gdf_ignosm_super.reset_index(inplace=True, drop=True)
gdf_ign['super_b'] = 0
gdf_osm['super_b'] = 0

for i in range(len(gdf_ignosm_super)):
    if gdf_ignosm_super.loc[i, "area_ign"] > gdf_ignosm_super.loc[i, "area_osm"]:
        gdf_ign.loc[gdf_ign.gid == gdf_ignosm_super.loc[i, "gid"], 'super_b'] = 1
    else:
        gdf_osm.loc[gdf_osm.osm_id == gdf_ignosm_super.loc[i, "osm_id"], 'super_b'] = 1

gdf_ign.loc[gdf_ign.super_b == 1].to_file("./Capas_Output/IGN_intersect.geojson", driver="GeoJSON")
gdf_osm.loc[gdf_osm.super_b == 1].to_file("./Capas_Output/OSM_intersect.geojson", driver="GeoJSON")
print("Polígonos IGN:", len(gdf_ign.loc[gdf_ign.super_b == 1]))
print("Polígonos OSM:", len(gdf_osm.loc[gdf_osm.super_b == 1]))
print("Polígonos Totales:", len(gdf_ign.loc[gdf_ign.super_b == 1])+len(gdf_osm.loc[gdf_osm.super_b == 1])) """

Polígonos IGN: 38
Polígonos OSM: 83
Polígonos Totales: 121


In [191]:
gdf_ignosm_super = gdf_ignosm_sjoin.loc[(gdf_ignosm_sjoin.index_right.notnull())]
gdf_ignosm_super.reset_index(inplace=True, drop=True)
gdf_ign['super_b'] = 0
gdf_osm['super_b'] = 0

for i in range(len(gdf_ignosm_super)):
    if gdf_ignosm_super.loc[i, "area_osm"] > gdf_ignosm_super.loc[i, "area_ign"]:
        gdf_osm.loc[gdf_osm.osm_id == gdf_ignosm_super.loc[i, "osm_id"], 'super_b'] = 1
    else:
        gdf_ign.loc[gdf_ign.gid == gdf_ignosm_super.loc[i, "gid"], 'super_b'] = 1

gdf_ign.loc[gdf_ign.super_b == 1].to_file("./Capas_Output/IGN_intersect.geojson", driver="GeoJSON")
gdf_osm.loc[gdf_osm.super_b == 1].to_file("./Capas_Output/OSM_intersect.geojson", driver="GeoJSON")
print("Polígonos IGN:", len(gdf_ign.loc[gdf_ign.super_b == 1]))
print("Polígonos OSM:", len(gdf_osm.loc[gdf_osm.super_b == 1]))
print("Polígonos Totales:", len(gdf_ign.loc[gdf_ign.super_b == 1])+len(gdf_osm.loc[gdf_osm.super_b == 1]))

Polígonos IGN: 81
Polígonos OSM: 40
Polígonos Totales: 121


# Unión de capas

In [192]:
# Unión de capas
gdf_osm_inter = gpd.read_file("./Capas_Output/OSM_intersect.geojson")
gdf_osm_inter['fuente'] = 'OSM'
gdf_ign_inter = gpd.read_file("./Capas_Output/IGN_intersect.geojson")
gdf_ign_inter['fuente'] = 'IGM'
gdf_osm_sininter = gpd.read_file("./Capas_Output/OSM_sinintersect.geojson")
gdf_osm_sininter['fuente'] = 'OSM'
gdf_ign_sininter = gpd.read_file("./Capas_Output/IGN_sinintersect.geojson")
gdf_ign_sininter['fuente'] = 'IGM'
gdf_evp = pd.concat([gdf_osm_inter, gdf_ign_inter, gdf_osm_sininter, gdf_ign_sininter])

In [193]:
gdf_evp.loc[gdf_evp.full_id.isnull(), 'full_id'] = gdf_evp.loc[gdf_evp.full_id.isnull(), 'gid']
gdf_evp.loc[gdf_evp.fna.isnull(), 'fna'] = gdf_evp.loc[gdf_evp.fna.isnull(), 'name']
gdf_evp.loc[gdf_evp.categoria.isnull(), 'categoria'] = 'park'
gdf_evp.loc[gdf_evp.limite.isnull(), 'limite'] = gdf_evp.loc[gdf_evp.limite.isnull(), 'limite_left']

In [194]:
gdf_evp_clean = gdf_evp[['full_id', 'fuente', 'DISTRITO', 'categoria', 'gna', 'fna', 'superpuesto', 'limite', 'geometry']]
gdf_evp_clean.reset_index(inplace=True, drop=True)
gdf_evp_clean.rename(columns={'DISTRITO':'distrito', 'gna':'tipo', 'fna':'nombre'}, inplace=True)
gdf_evp_clean.head()

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\pandas\core\frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,full_id,fuente,distrito,categoria,tipo,nombre,superpuesto,limite,geometry
0,r2895800,OSM,LOMAS DE ZAMORA,park,NaN,Plaza San Martín,1.0,0.0,"MULTIPOLYGON (((-58.40580 -34.79124, -58.40457..."
1,r5514857,OSM,LANUS,park,NaN,,1.0,1.0,"MULTIPOLYGON (((-58.39026 -34.73278, -58.39026..."
2,r13232486,OSM,LOMAS DE ZAMORA,park,NaN,Plaza Carlos Pellegrini,0.0,1.0,"MULTIPOLYGON (((-58.42685 -34.79316, -58.42684..."
3,w28051684,OSM,LOMAS DE ZAMORA,park,NaN,Plaza Libertad,0.0,1.0,"MULTIPOLYGON (((-58.41578 -34.75867, -58.41435..."
4,w28051696,OSM,LOMAS DE ZAMORA,park,NaN,Plaza de los Italianos,0.0,1.0,"MULTIPOLYGON (((-58.42691 -34.76192, -58.42614..."


In [196]:
print(len(gdf_evp_clean))

233


In [204]:
# completar datos de nombre y tipo según IGN
gdf_esping_sjoin = gpd.sjoin(gdf_evp_clean, gdf_ign[['fna', 'gna', 'geometry']], how='left')
gdf_esping_sjoin.loc[gdf_esping_sjoin.fna.notnull(), 'nombre'] = gdf_esping_sjoin.loc[gdf_esping_sjoin.fna.notnull(), 'fna']
gdf_esping_sjoin.loc[gdf_esping_sjoin.gna.notnull(), 'tipo'] = gdf_esping_sjoin.loc[gdf_esping_sjoin.gna.notnull(), 'gna']
gdf_esping_sjoin = gdf_esping_sjoin[['full_id', 'fuente', 'distrito', 'categoria', 'tipo', 'nombre', 'superpuesto', 'limite', 'geometry']]
gdf_esping_sjoin

,full_id,fuente,distrito,categoria,tipo,nombre,superpuesto,limite,geometry
0,r2895800,OSM,LOMAS DE ZAMORA,park,Plaza,Plaza General San Martín,1.0,0.0,"MULTIPOLYGON (((-58.40580 -34.79124, -58.40457..."
1,r5514857,OSM,LANUS,park,Plaza,Plaza Héroes de Malvinas,1.0,1.0,"MULTIPOLYGON (((-58.39026 -34.73278, -58.39026..."
1,r5514857,OSM,LANUS,park,NaN,,1.0,1.0,"MULTIPOLYGON (((-58.39026 -34.73278, -58.39026..."
2,r13232486,OSM,LOMAS DE ZAMORA,park,NaN,Plaza Carlos Pellegrini,0.0,1.0,"MULTIPOLYGON (((-58.42685 -34.79316, -58.42684..."
3,w28051684,OSM,LOMAS DE ZAMORA,park,Plaza,Plaza Libertad,0.0,1.0,"MULTIPOLYGON (((-58.41578 -34.75867, -58.41435..."
...,...,...,...,...,...,...,...,...,...
228,1511,IGM,LOMAS DE ZAMORA,park,Parque,Parque de Recreación Nº 1,NaN,1.0,"MULTIPOLYGON (((-58.45592 -34.69934, -58.45704..."
229,3532,IGM,LOMAS DE ZAMORA,park,Plaza,Plaza Comunal,NaN,1.0,"MULTIPOLYGON (((-58.44989 -34.71411, -58.44934..."
230,5801,IGM,LOMAS DE ZAMORA,park,None,NaN,NaN,1.0,"MULTIPOLYGON (((-58.40974 -34.77363, -58.40958..."
231,5970,IGM,LOMAS DE ZAMORA,park,None,NaN,NaN,0.0,"MULTIPOLYGON (((-58.37626 -34.75415, -58.37612..."


In [205]:
# Eliminar duplicados
gdf_esping_sjoin.drop_duplicates(subset='full_id', keep='first', inplace=True, ignore_index=True)

In [230]:
# Registro superpuestos entre si
gdf_esping_sjoin['superpuesto'] = 0

for i in range(len(gdf_esping_sjoin)):
    gdf_registrosuperpuesto = gpd.sjoin(gdf_esping_sjoin.loc[[i]], gdf_esping_sjoin.loc[gdf_esping_sjoin.index != i], how='left')

    if len(gdf_registrosuperpuesto.loc[gdf_registrosuperpuesto.full_id_right.notnull()]) > 0:
        gdf_esping_sjoin.loc[i, 'superpuesto'] = 1

print("Polígonos sin superposición", len(gdf_esping_sjoin.loc[gdf_esping_sjoin['superpuesto']==0]))
print("Polígonos con superposición", len(gdf_esping_sjoin.loc[gdf_esping_sjoin['superpuesto']==1]))

Polígonos sin superposición 216
Polígonos con superposición 17


In [247]:
gdf_esping_sjoin.loc[gdf_esping_sjoin.tipo == 'Parque Municipal', 'tipo'] = 'Parque'

gdf_esping_sjoin['evp'] = 0
gdf_esping_sjoin.loc[gdf_esping_sjoin.categoria == 'park', 'evp'] = 1
gdf_esping_sjoin = gdf_esping_sjoin[['full_id', 'fuente', 'distrito', 'evp', 'categoria', 'tipo', 'nombre', 'superpuesto', 'limite', 'geometry']]
gdf_esping_sjoin

,full_id,fuente,distrito,evp,categoria,tipo,nombre,superpuesto,limite,geometry
0,r2895800,OSM,LOMAS DE ZAMORA,1,park,Plaza,Plaza General San Martín,1,0.0,"MULTIPOLYGON (((-58.40580 -34.79124, -58.40457..."
1,r5514857,OSM,LANUS,1,park,Plaza,Plaza Héroes de Malvinas,1,1.0,"MULTIPOLYGON (((-58.39026 -34.73278, -58.39026..."
2,r13232486,OSM,LOMAS DE ZAMORA,1,park,NaN,Plaza Carlos Pellegrini,0,1.0,"MULTIPOLYGON (((-58.42685 -34.79316, -58.42684..."
3,w28051684,OSM,LOMAS DE ZAMORA,1,park,Plaza,Plaza Libertad,0,1.0,"MULTIPOLYGON (((-58.41578 -34.75867, -58.41435..."
4,w28051696,OSM,LOMAS DE ZAMORA,1,park,NaN,Plaza de los Italianos,0,1.0,"MULTIPOLYGON (((-58.42691 -34.76192, -58.42614..."
...,...,...,...,...,...,...,...,...,...,...
228,1511,IGM,LOMAS DE ZAMORA,1,park,Parque,Parque de Recreación Nº 1,0,1.0,"MULTIPOLYGON (((-58.45592 -34.69934, -58.45704..."
229,3532,IGM,LOMAS DE ZAMORA,1,park,Plaza,Plaza Comunal,0,1.0,"MULTIPOLYGON (((-58.44989 -34.71411, -58.44934..."
230,5801,IGM,LOMAS DE ZAMORA,1,park,None,NaN,0,1.0,"MULTIPOLYGON (((-58.40974 -34.77363, -58.40958..."
231,5970,IGM,LOMAS DE ZAMORA,1,park,None,NaN,0,0.0,"MULTIPOLYGON (((-58.37626 -34.75415, -58.37612..."


In [248]:
# Exportar capa
gdf_esping_sjoin.to_file("./Capas_Output/EspaciosPublicos.geojson", driver="GeoJSON")

# Cálculo de superficies

In [18]:
# Importar capa
gdf = gpd.read_file("./Capas_Output/EspaciosPublicos_Pau.geojson")
gdf = gdf.to_crs("EPSG:5347")
gdf['area_m2'] = gdf.area
gdf.area_m2 = round(gdf.area_m2, 2)
gdf = gdf.to_crs("EPSG:4326")
gdf.rename(columns={'Observaciones':'observaciones'}, inplace=True)
gdf = gdf[['full_id', 'fuente', 'distrito', 'evp', 'categoria', 'tipo', 'nombre','superpuesto', 'limite', 'area_m2', 'observaciones', 'geometry']]
gdf.head()

,full_id,fuente,distrito,evp,categoria,tipo,nombre,superpuesto,limite,area_m2,observaciones,geometry
0,r2895800,OSM,LOMAS DE ZAMORA,1.0,park,Plaza,Plaza General San Martín,1.0,0.0,9069.92,None,"MULTIPOLYGON (((-58.40580 -34.79124, -58.40457..."
1,r5514857,OSM,LANUS,1.0,park,Plaza,Plaza Héroes de Malvinas,1.0,1.0,167085.78,None,"MULTIPOLYGON (((-58.39026 -34.73278, -58.39026..."
2,r13232486,OSM,LOMAS DE ZAMORA,1.0,park,None,Plaza Carlos Pellegrini,0.0,1.0,6932.85,None,"MULTIPOLYGON (((-58.42685 -34.79316, -58.42684..."
3,w28051684,OSM,LOMAS DE ZAMORA,1.0,park,Plaza,Plaza Libertad,0.0,1.0,14263.38,None,"MULTIPOLYGON (((-58.41578 -34.75867, -58.41435..."
4,w28051696,OSM,LOMAS DE ZAMORA,1.0,park,None,Plaza de los Italianos,0.0,1.0,7883.71,None,"MULTIPOLYGON (((-58.42691 -34.76192, -58.42614..."


In [19]:
# corrección de superpuestos
# eliminar superpuestos que no deberían estar
print(len(gdf))
lista_e = ['w149700227', 'r5514857', 'w241322368', 'r2895800', '5902']
for e in lista_e:
    gdf.drop(gdf.loc[gdf.full_id == e].index, inplace=True)
gdf.reset_index(inplace=True, drop=True)
print(len(gdf), '-', len(lista_e))
# modificar valor del campo superpuesto para los polígonos que quedan
for s in ['1890', '12271', '6609', '5828', 'w167029793', 'w28184879', 'w224939240', 'w291374429', 'w302130495', 'w224939237', 'w224939239', 'w224939242']:
    gdf.loc[gdf.full_id == s, 'superpuesto'] = 0

235
230 - 5


In [20]:
# corregir polígono con error
gdf.loc[87, 'full_id'] = '10367'
gdf.loc[gdf.full_id == '10367', 'fuente'] = 'IGN'
gdf.loc[gdf.full_id == '10367', 'distrito'] = 'LANUS'
gdf.loc[gdf.full_id == '10367', 'categoria'] = 'park'
gdf.loc[gdf.full_id == '10367', 'limite'] = 0
gdf.loc[gdf.full_id == '10367']

,full_id,fuente,distrito,evp,categoria,tipo,nombre,superpuesto,limite,area_m2,observaciones,geometry
87,10367,IGN,LANUS,1.0,park,Plazoleta,Plazoleta Berón de Astrada,0.0,0.0,929.78,,"MULTIPOLYGON (((-58.37872 -34.72369, -58.37845..."


In [21]:
# eliminar polígonos que no corresponden por ID
print(len(gdf))
lista_e2 = ['13221', '13562', 'w678895041']
for e in lista_e2:   # 'w678895041' => 10367
    gdf.drop(gdf.loc[gdf.full_id == e].index, inplace=True)
gdf.reset_index(inplace=True, drop=True)
print(len(gdf), '-', len(lista_e2))

230
229 - 3


In [22]:
# eliminar polígonos sin datos
print(len(gdf))
gdf = gdf.loc[gdf.fuente.notnull()] 
gdf.reset_index(inplace=True, drop=True)
print(len(gdf))

229
225


In [25]:
# Corrección de fuente
gdf.loc[gdf.fuente == 'IGM', 'fuente'] = 'IGN'

In [26]:
gdf.fuente.value_counts()

OSM      142
IGN       82
UNSAM      1
Name: fuente, dtype: int64

In [27]:
# exportar capa
gdf.to_file("./Capas_Output/EspaciosPublicos_v2.geojson", driver="GeoJSON")